In [3]:
from __future__ import print_function
import keras as keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import matplotlib.pyplot as plt

print("hej")
############ Initialize variables ######
nInputs = 21
nOutputs = 7

#data = np.genfromtxt('Sarcos_csv.csv', delimiter=',')

hidden_layers = 256
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.

############ Loading data ############## 
# load csv data for input and output

# 7 columns each for joint angles, angular velocities and angular acceleraions
uVec = pd.read_csv("Sarcos_csv.csv", usecols=np.arange(21),  header=None,
                  names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 
                           'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7'])
# 7 columns for joint torques
yVec = pd.read_csv("Sarcos_csv.csv", usecols=np.arange(21, 28, 1), header=None, 
                  names = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7'])

# to numpy array
u = uVec.values[:, :, np.newaxis]
y = yVec.values[:, :]#, np.newaxis]
print("shape_y:",y.shape)
split = 0.5
ind = round(split*u.shape[0])
print("ind:",ind)
x_train = u[0:ind,0:nInputs]#,:]
y_train = y[0:ind,0:]#,np.newaxis]#1:2] 
x_test = u[ind:,0:nInputs]#,:]
y_test = y[ind:,0:]#,np.newaxis]

#x_train = u[0:ind,0:nInputs,np.newaxis]
#y_train = y[0:ind,nInputs:]#,np.newaxis]#1:2] 
#x_test = u[ind:,0:nInputs,np.newaxis]
#y_test = y[ind:,nInputs:]#,np.newaxis]

print("x_train size: {}".format(x_train.shape))
print("y_train size: {}".format(y_train.shape))
print("x_test size: {}".format(x_test.shape))
print("y_test size: {}".format(y_test.shape))


####### 
model = Sequential()

model.add(Dense(nInputs,input_shape=(21,1))) #correct with nInputs?
model.add(LSTM(hidden_layers,return_sequences=False))

model.add(Dense(nOutputs))
model.compile(loss='mse',optimizer ='adam',metrics=['accuracy'])




hej
shape_y: (44484, 7)
ind: 22242
x_train size: (22242, 21, 1)
y_train size: (22242, 7)
x_test size: (22242, 21, 1)
y_test size: (22242, 7)


In [ ]:
model.fit(x_train,y_train,epochs=2000,batch_size=5,validation_split=0.05,verbose=0);


scores = model.evaluate(X,y,verbose=1,batch_size=5)
print('Accurracy: {}'.format(scores[1])) 

predict=model.predict(X)
plt.plot(y, predict-y, 'C2')
plt.ylim(ymax = 3, ymin = -3)
plt.show()

In [ ]:
keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                  kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                  bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
                  recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
                  kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
                  dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, 
                  return_state=False, go_backwards=False, stateful=False, unroll=False)




In [ ]:





model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

############ Further configuration of optimizer ############# 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))



####################################################################################

input_characters = x_train[:,1] #change this
print("input_characters:",input_characters)
num_encoder_tokens = len(input_characters)
print("num_encoder_tokens:",num_encoder_tokens)
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using 'encoder_states' as initial state.
target_characters = y_train[0] #change this
num_decoder_tokens = len(target_characters)
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
#model.fit(x_train, y_train,
          #batch_size=batch_size,
          #epochs=epochs,
          #validation_split=0.2)
#model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          #batch_size=batch_size,
          #epochs=epochs,
          #validation_split=0.2)